# SplitComponentsImageFilter

This notebook demonstrates how to use `itk.SplitComponentsImageFilter` to decompose an image into its constituent channels.

An RGB image is decomposed into grayscale images representing Red, Green, and Blue channels. The image is then reassembled with an alpha channel to form an RGBA image for viewing.

In [1]:
import itk
from itkwidgets import view
from ipywidgets import interactive
import ipywidgets as widgets
import urllib.request

In [2]:
# Download image
urllib.request.urlretrieve("https://raw.githubusercontent.com/InsightSoftwareConsortium/ITK/master/Examples/Data/VisibleWomanHeadSlice.png", "VisibleWomanHeadSlice.png")

('VisibleWomanHeadSlice.png', <http.client.HTTPMessage at 0x7f812a937970>)

In [3]:
# Read image
image = itk.imread("VisibleWomanHeadSlice.png")
print("Image Type: %s" % str(itk.template(image)))
print("Image Pixel Type: %s" % str(itk.template(itk.template(image)[1][0])))
view(image)

Image Type: (<itkTemplate itk::Image>, (<class 'itk.itkRGBPixelPython.itkRGBPixelUC'>, 2))
Image Pixel Type: (<itkTemplate itk::RGBPixel>, (<itkCType unsigned char>,))


Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageRGBUC2;…

In [4]:
# Decompose image in 3 components: R, G, and B
split_images = itk.split_components_image_filter(image)

rgb_viewer = view(image, annotations=False)
viewer = view(split_images[0], cmap="gray_Matlab", annotations=False)

# Define a function to use with ipywidgets `interactive`
def view_channels(index=0):
    # Update the image used in the viewer
    viewer.image = split_images[index]
slider = interactive(view_channels, index=(0, len(split_images)-1, 1))

widgets.HBox([rgb_viewer, widgets.VBox([viewer, slider])])

In [5]:
print("Blue Image Information: %s" % str(split_images[2]))

Blue Image Information: Image (0x55e1e2d520a0)
  RTTI typeinfo:   itk::Image<unsigned char, 2u>
  Reference Count: 1
  Modified Time: 454
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 435
  UpdateMTime: 451
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [142, 167]
  BufferedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [142, 167]
  RequestedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [142, 167]
  Spacing: [1, 1]
  Origin: [0, 0]
  Direction: 
1 0
0 1

  IndexToPointMatrix: 
1 0
0 1

  PointToIndexMatrix: 
1 0
0 1

  Inverse Direction: 
1 0
0 1

  PixelContainer: 
    ImportImageContainer (0x55e1e67a4330)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, unsigned char>
      Reference Count: 1
      Modified Time: 445
      Debug: Off
      Object Name: 
      Observe

In [10]:
# Recompose image with an Alpha layer.
RGBAImageType = itk.Image[itk.RGBAPixel[itk.UC], image.GetImageDimension()]
compose_image_filter = itk.ComposeImageFilter[split_images[0], RGBAImageType].New()
compose_image_filter.SetInput(0, split_images[0])
compose_image_filter.SetInput(1, split_images[1])
compose_image_filter.SetInput(2, split_images[2])

alpha_layer = itk.image_duplicator(split_images[2])
alpha_layer.FillBuffer(255)
compose_image_filter.SetInput(3, alpha_layer)

compose_image_filter.Update()
recomposed_image = compose_image_filter.GetOutput()
print(recomposed_image)
view(recomposed_image)

Image (0x55e1e6801270)
  RTTI typeinfo:   itk::Image<itk::RGBAPixel<unsigned char>, 2u>
  Reference Count: 2
  Modified Time: 568
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (0x55e1e37d4640) 
  Source output name: Primary
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 563
  UpdateMTime: 569
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [142, 167]
  BufferedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [142, 167]
  RequestedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [142, 167]
  Spacing: [1, 1]
  Origin: [0, 0]
  Direction: 
1 0
0 1

  IndexToPointMatrix: 
1 0
0 1

  PointToIndexMatrix: 
1 0
0 1

  Inverse Direction: 
1 0
0 1

  PixelContainer: 
    ImportImageContainer (0x55e1e464dfe0)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, itk::RGBAPixel<unsigned char> >
      Reference Count: 1
      Modified Time: 567
      Debug: Off
      Object

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageRGBAUC2…